In [ ]:
# import pandas as pd
# ts=pd.read_csv(open('./matchbymatch.csv','rb'))
# teams={}
# for i in range(30):
#     teams[ts['Team'][i]]=0
# for i in range(30):
#     secret_metric=((ts['NetRtg'][i]*10)+ts['FG%'][i]+ts['FT%'][i]+ts['3PT%'][i]+ts['ORB%'][i]+\
#                                                          ts['Assist%'][i]+ts['Wins'][i]+(ts['PtDiff'][i]/10))*22/7
#     teams[ts['Team'][i]] = secret_metric

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
final_team_stats = pd.read_csv(open('./matchbymatch.csv','rb'), index_col = 0)[::-1]
# final_team_stats = final_team_stats.reindex(index=final_team_stats.index[::-1])
# print(final_team_stats.head())

cols =  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,33,34,37,38]
final_team_stats.drop(final_team_stats.columns[cols],axis=1, inplace=True)
print(final_team_stats.head())

In [ ]:
features = final_team_stats.drop(columns = 'Winner')
label = final_team_stats['Winner']

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pickle

features = final_team_stats.drop(columns = 'Winner')
label = final_team_stats['Winner']

max_acc=0
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2)

    # create complex Logistic Regression with max_iter=160
    log_model = make_pipeline(StandardScaler(), LogisticRegression())
    log_model.fit(x_train, y_train)
    y_pred_log = log_model.predict(x_test)
    acc= float(metrics.accuracy_score(y_test, y_pred_log))

    if acc > max_acc :
        pickle.dump(log_model, open('prediction.pkl', 'wb'))
        max_acc=acc

print("accuracy is: "+"{:.2f}".format(max_acc*100)+"%")

In [ ]:
loaded_model = pickle.load(open('prediction.pkl', 'rb'))
y_pred = loaded_model.predict(x_test)
acc= float(metrics.accuracy_score(y_test, y_pred))

print('logistic regression : '+"{:.2f}".format(acc*100)+"%")

In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)

acc= float(metrics.accuracy_score(y_test, y_pred_rf))

print('random forest : '+"{:.2f}".format(acc*100)+"%")

In [ ]:
# linear svc
from sklearn.svm import LinearSVC

lclf = LinearSVC(random_state=0, tol=1e-5, max_iter=1000)
lclf.fit(x_train, y_train)
y_pred_svc = lclf.predict(x_test)

acc= float(metrics.accuracy_score(y_test, y_pred_svc))

print('linear svc : '+"{:.2f}".format(acc*100)+"%")

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test)

acc= float(metrics.accuracy_score(y_test, y_pred_knn))

print('KNN : '+"{:.2f}".format(acc*100)+"%")

In [ ]:

import pandas as pd
class nbaWinPredictor:
    def recent_performance(Home,Away):
        Home=Home.upper()
        Away=Away.upper()
        gl=pd.read_csv(open('./matchbymatch.csv','rb'), index_col = 0)
        h_win=0
        a_win=0
        for(i,j,k) in zip(gl['Home'], gl['Away'], gl['Winner']):
            if(i==Home and j==Away):
                if(k=="H"):
                    h_win+=1
                else:
                    a_win+=1
            if(i==Away and j==Home):
                if(k=="H"):
                    a_win+=1
                else:
                    h_win+=1
        return 'In the 2017-18 NBA season, when these two teams played each other, '+  Home + " Won "+ str(h_win) + " times, while "+ Away + " won "+ str(a_win)+" times"
            
    def prediction_model(Home,Away):
        Home=Home.upper()
        Away=Away.upper()
        match_by_match_stats = pd.read_csv(open('./matchbymatch.csv','rb'), index_col = 0)[::-1]
        total_no_of_matches = len(match_by_match_stats)
        cols=[]
        home_stats=[]
        away_stats=[]
        home_elo=0
        away_elo=0
        home_sm=0
        away_sm=0
        home_found=False
        away_found=False
        col_names=['HRFG%','HR3P%','HRFT%','HRO','HRT','HRS','HRP','ARFG%','AR3P%','ARFT%','ARO','ART','ARS','ARP','Home_elo_before','Away_elo_before','Home_sm_before','Away_sm_before']
        for i in range(total_no_of_matches-1,0,-1):
            if(match_by_match_stats['Home'][i]==Home and not home_found): 
                home_found=True
                home_elo=match_by_match_stats['Home_elo_before'][i]
                home_sm=match_by_match_stats['Home_sm_before'][i]
                home_stats.append(match_by_match_stats['HRFG%'][i])
                home_stats.append(match_by_match_stats['HR3P%'][i])
                home_stats.append(match_by_match_stats['HRFT%'][i])
                home_stats.append(match_by_match_stats['HRO'][i])
                home_stats.append(match_by_match_stats['HRT'][i])
                home_stats.append(match_by_match_stats['HRS'][i])
                home_stats.append(match_by_match_stats['HRP'][i])
            if(match_by_match_stats['Away'][i]==Away and not away_found):
                away_found=True
                away_elo=match_by_match_stats['Away_elo_before'][i]
                away_sm=match_by_match_stats['Away_sm_before'][i]
                away_stats.append(match_by_match_stats['ARFG%'][i])
                away_stats.append(match_by_match_stats['AR3P%'][i])
                away_stats.append(match_by_match_stats['ARFT%'][i])
                away_stats.append(match_by_match_stats['ARO'][i])
                away_stats.append(match_by_match_stats['ART'][i])
                away_stats.append(match_by_match_stats['ARS'][i])
                away_stats.append(match_by_match_stats['ARP'][i])
            if(home_found and away_found):
                cols.extend(home_stats)
                cols.extend(away_stats)
                cols.extend([home_elo,away_elo,home_sm,away_sm])
                break
        df=pd.DataFrame([cols],columns=col_names)
        res="".join(log_model.predict(df))
        win=Home if res=='H' else Away
        return 'Team '+ win +' has a higher chance to win with an accuracy of '+"{:.2f}".format(acc*100)+'%'


In [ ]:
nbaWinPredictor.prediction_model('atl','gsw')